In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


[Learn the Basics](intro.html) ||
**Quickstart** ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
[Build Model](buildmodel_tutorial.html) ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Quickstart
This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

## Working with data
PyTorch has two [primitives to work with data](https://pytorch.org/docs/stable/data.html):
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html),
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like
CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element
in the dataloader iterable will return a batch of 64 features and labels.



In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](data_tutorial.html).




--------------




## Creating Models
To define a neural network in PyTorch, we create a class that inherits
from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network
in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate
operations in the neural network, we move it to the GPU or MPS if available.



In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in PyTorch](buildmodel_tutorial.html).




--------------




## Optimizing the Model Parameters
To train a model, we need a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions)
and an [optimizer](https://pytorch.org/docs/stable/optim.html).



In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and
backpropagates the prediction error to adjust the model's parameters.



In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Batch : {batch}")
        if batch % 100 == 0:
            print(f"Batch % 100 : {batch}")
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.



In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    print(f"size: {size}, num_batches:{num_batches}")
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            print("X.ndim", X.ndim)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
#             print("pred.argmax(1)", (pred.argmax(1) == y))
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            print(f"correct: {correct}")
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [9]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Batch : 0
Batch % 100 : 0
loss: 2.303708  [   64/60000]
Batch : 1
Batch : 2
Batch : 3
Batch : 4
Batch : 5
Batch : 6
Batch : 7
Batch : 8
Batch : 9
Batch : 10
Batch : 11
Batch : 12
Batch : 13
Batch : 14
Batch : 15
Batch : 16
Batch : 17
Batch : 18
Batch : 19
Batch : 20
Batch : 21
Batch : 22
Batch : 23
Batch : 24
Batch : 25
Batch : 26
Batch : 27
Batch : 28
Batch : 29
Batch : 30
Batch : 31
Batch : 32
Batch : 33
Batch : 34
Batch : 35
Batch : 36
Batch : 37
Batch : 38
Batch : 39
Batch : 40
Batch : 41
Batch : 42
Batch : 43
Batch : 44
Batch : 45
Batch : 46
Batch : 47
Batch : 48
Batch : 49
Batch : 50
Batch : 51
Batch : 52
Batch : 53
Batch : 54
Batch : 55
Batch : 56
Batch : 57
Batch : 58
Batch : 59
Batch : 60
Batch : 61
Batch : 62
Batch : 63
Batch : 64
Batch : 65
Batch : 66
Batch : 67
Batch : 68
Batch : 69
Batch : 70
Batch : 71
Batch : 72
Batch : 73
Batch : 74
Batch : 75
Batch : 76
Batch : 77
Batch : 78
Batch : 79
Batch : 80
Batch : 81
Batch : 82
Batch : 83


Batch : 685
Batch : 686
Batch : 687
Batch : 688
Batch : 689
Batch : 690
Batch : 691
Batch : 692
Batch : 693
Batch : 694
Batch : 695
Batch : 696
Batch : 697
Batch : 698
Batch : 699
Batch : 700
Batch % 100 : 700
loss: 2.199369  [44864/60000]
Batch : 701
Batch : 702
Batch : 703
Batch : 704
Batch : 705
Batch : 706
Batch : 707
Batch : 708
Batch : 709
Batch : 710
Batch : 711
Batch : 712
Batch : 713
Batch : 714
Batch : 715
Batch : 716
Batch : 717
Batch : 718
Batch : 719
Batch : 720
Batch : 721
Batch : 722
Batch : 723
Batch : 724
Batch : 725
Batch : 726
Batch : 727
Batch : 728
Batch : 729
Batch : 730
Batch : 731
Batch : 732
Batch : 733
Batch : 734
Batch : 735
Batch : 736
Batch : 737
Batch : 738
Batch : 739
Batch : 740
Batch : 741
Batch : 742
Batch : 743
Batch : 744
Batch : 745
Batch : 746
Batch : 747
Batch : 748
Batch : 749
Batch : 750
Batch : 751
Batch : 752
Batch : 753
Batch : 754
Batch : 755
Batch : 756
Batch : 757
Batch : 758
Batch : 759
Batch : 760
Batch : 761
Batch : 762
Batch : 763
Batc

Batch : 104
Batch : 105
Batch : 106
Batch : 107
Batch : 108
Batch : 109
Batch : 110
Batch : 111
Batch : 112
Batch : 113
Batch : 114
Batch : 115
Batch : 116
Batch : 117
Batch : 118
Batch : 119
Batch : 120
Batch : 121
Batch : 122
Batch : 123
Batch : 124
Batch : 125
Batch : 126
Batch : 127
Batch : 128
Batch : 129
Batch : 130
Batch : 131
Batch : 132
Batch : 133
Batch : 134
Batch : 135
Batch : 136
Batch : 137
Batch : 138
Batch : 139
Batch : 140
Batch : 141
Batch : 142
Batch : 143
Batch : 144
Batch : 145
Batch : 146
Batch : 147
Batch : 148
Batch : 149
Batch : 150
Batch : 151
Batch : 152
Batch : 153
Batch : 154
Batch : 155
Batch : 156
Batch : 157
Batch : 158
Batch : 159
Batch : 160
Batch : 161
Batch : 162
Batch : 163
Batch : 164
Batch : 165
Batch : 166
Batch : 167
Batch : 168
Batch : 169
Batch : 170
Batch : 171
Batch : 172
Batch : 173
Batch : 174
Batch : 175
Batch : 176
Batch : 177
Batch : 178
Batch : 179
Batch : 180
Batch : 181
Batch : 182
Batch : 183
Batch : 184
Batch : 185
Batch : 186
Batc

Batch : 763
Batch : 764
Batch : 765
Batch : 766
Batch : 767
Batch : 768
Batch : 769
Batch : 770
Batch : 771
Batch : 772
Batch : 773
Batch : 774
Batch : 775
Batch : 776
Batch : 777
Batch : 778
Batch : 779
Batch : 780
Batch : 781
Batch : 782
Batch : 783
Batch : 784
Batch : 785
Batch : 786
Batch : 787
Batch : 788
Batch : 789
Batch : 790
Batch : 791
Batch : 792
Batch : 793
Batch : 794
Batch : 795
Batch : 796
Batch : 797
Batch : 798
Batch : 799
Batch : 800
Batch % 100 : 800
loss: 1.982932  [51264/60000]
Batch : 801
Batch : 802
Batch : 803
Batch : 804
Batch : 805
Batch : 806
Batch : 807
Batch : 808
Batch : 809
Batch : 810
Batch : 811
Batch : 812
Batch : 813
Batch : 814
Batch : 815
Batch : 816
Batch : 817
Batch : 818
Batch : 819
Batch : 820
Batch : 821
Batch : 822
Batch : 823
Batch : 824
Batch : 825
Batch : 826
Batch : 827
Batch : 828
Batch : 829
Batch : 830
Batch : 831
Batch : 832
Batch : 833
Batch : 834
Batch : 835
Batch : 836
Batch : 837
Batch : 838
Batch : 839
Batch : 840
Batch : 841
Batc

In [ ]:
model.state_dict()

Read more about [Training your model](optimization_tutorial.html).




--------------




## Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [16]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it.



In [17]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.



In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"


Read more about [Saving & Loading your model](saveloadrun_tutorial.html).


